# How likely is a Bones Day vs. a No Bones Day?

In [1]:
%load_ext lab_black

### Load Python tools

In [2]:
import pandas as pd
import altair as alt
import datetime as dt

In [3]:
today = dt.datetime.today()

### Read data collected from [TikTok](https://www.tiktok.com/@jongraz) about Noodle

In [4]:
src = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQz8eEafXnIeKsNvGL8M56Ia6vu89JDCjfJ22ORl65So4kRYtmTbwmqLjE223fe4mjcZqkrs-KT6j8z/pub?\
gid=0&single=true&output=csv"
)

### Process dates

In [5]:
src["date"] = pd.to_datetime(src["date"])

In [6]:
df_date = pd.DataFrame()
index = pd.date_range(start="2021-09-01", end=today, freq="D")
df_date["date"] = index

### Merge the dates to backfill

In [7]:
df = pd.merge(src, df_date, on="date", how="outer")

In [8]:
df["weekday"] = df["date"].dt.day_name()
df["month"] = df["date"].dt.month_name()
df["weekend"] = df["date"].dt.day_name().isin(["Saturday", "Sunday"])

In [9]:
df.drop(["url"], axis=1, inplace=True)

In [10]:
df.bones_nobones = df.bones_nobones.fillna("No video")

### What's our dataframe look like? 

In [11]:
df.head()

,date,bones_nobones,weekday,month,weekend
0,2021-11-18,Bones!,Thursday,November,False
1,2021-11-16,Bones!,Tuesday,November,False
2,2021-11-15,Bones!,Monday,November,False
3,2021-11-12,No bones,Friday,November,False
4,2021-11-11,No bones,Thursday,November,False


### Percentage of days with Bones (only days with a video)

In [12]:
(
    df[df["bones_nobones"] != "No video"].value_counts("bones_nobones", normalize=True)
    * 100
).round(0)

bones_nobones
Bones!      50.0
No bones    50.0
dtype: float64

### Group by day type

In [13]:
days = (
    df.groupby(["bones_nobones"])
    .agg({"date": "count"})
    .reset_index()
    .rename(columns={"date": "count"})
)

### Simple bar chart explaining the count of day types

In [14]:
alt.Chart(days[days["bones_nobones"] != "No video"], title=" ").mark_bar().encode(
    y=alt.Y("bones_nobones", title="", axis=alt.Axis(tickCount=0, tickColor="#ffffff")),
    x=alt.X(
        "count",
        title="Days",
        axis=alt.Axis(grid=False, tickCount=6, tickColor="#ffffff"),
    ),
    color=alt.Color(
        "bones_nobones",
        title="",
        legend=None,
        scale=alt.Scale(
            domain=["Bones!", "No bones", "No video"],
            range=["#5ab4ac", "#d8b365", "#e6e6e6"],
        ),
    ),
).properties(width=500, height=70)

alt.Chart(...)

### Plot Noodle's days on a calendar

In [15]:
heatmap = (
    alt.Chart(df[df["date"] > "2021-08-31"], title=" ")
    .mark_rect()
    .encode(
        x=alt.X("date(date):O", title=" "),
        y=alt.Y("month(date):O", title=""),
        color=alt.Color(
            "bones_nobones",
            title="",
            scale=alt.Scale(
                domain=["Bones!", "No bones", "No video"],
                range=["#5ab4ac", "#d8b365", "#e6e6e6"],
            ),
        ),
        tooltip=[
            alt.Tooltip("monthdate(date):T", title="Date"),
            alt.Tooltip("bones_nobones", title="Tweets"),
        ],
    )
    .properties(width=500, height=70, title="Bones days since Sept. 1")
)

heatmap.configure_legend(orient="top", padding=10).configure_view(strokeOpacity=0)

alt.Chart(...)

### Prep for datawrapper 

In [48]:
df.head()

,date,bones_nobones,weekday,month,weekend
0,2021-11-18,Bones!,Thursday,November,False
1,2021-11-16,Bones!,Tuesday,November,False
2,2021-11-15,Bones!,Monday,November,False
3,2021-11-12,No bones,Friday,November,False
4,2021-11-11,No bones,Thursday,November,False


In [49]:
grouped_all = (
    df.groupby("bones_nobones")
    .agg({"date": "size"})
    .reset_index()
    .rename(columns={"date": "count"})
)

In [45]:
grouped_weekends = (
    df.groupby(["bones_nobones", "weekend"])
    .agg({"date": "size"})
    .reset_index()
    .rename(columns={"date": "count"})
)

In [46]:
grouped_weekends

,bones_nobones,weekend,count
0,Bones!,False,17
1,Bones!,True,3
2,No bones,False,15
3,No bones,True,5
4,No video,False,28
5,No video,True,15


In [50]:
grouped_all.drop(2, axis=0, inplace=True)
grouped_weekends.drop([4, 5], axis=0, inplace=True)

In [58]:
pivot_all = pd.pivot_table(grouped, values="count", columns="bones_nobones")
pivot_weekend = pd.pivot_table(
    grouped, values="count", columns=["weekend"]
)

KeyError: 'weekend'

In [56]:
pivot_all

bones_nobones,Bones!,No bones
count,20,20


In [57]:
pivot_weekend

,bones_nobones,weekend
count,20,20


### Export

In [42]:
pivot.to_csv("data/processed/bones_stacked_bar.csv")
df.to_csv("data/processed/bones_nobones.csv", index=False)